In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# Load OA papers that cite AF levels 0-3
# Currently using no_mesh
# When mesh has been added use "oa.data_processing.depth.mesh.{level}.intermediate" (make note in PR)
level_0_primary = catalog.load("oa.data_processing.depth.no_mesh.0.intermediate")
level_1_primary = catalog.load("oa.data_processing.depth.no_mesh.1.intermediate")
level_2_primary = catalog.load("oa.data_processing.depth.no_mesh.2.intermediate")
level_3_primary = catalog.load("oa.data_processing.depth.no_mesh.3.intermediate")

                    INFO     Loading data from oa.data_processing.depth.no_mesh.0.intermediate  ]8;id=773188;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=45007;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (ParquetDataset)...                                                                   

[03/21/24 14:53:15] INFO     Loading data from oa.data_processing.depth.no_mesh.1.intermediate  ]8;id=351704;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=441170;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (ParquetDataset)...                                                                   

[03/21/24 14:53:16] INFO     Loading data from oa.data_processing.depth.no_mesh.2.intermediate  ]8;id=574074;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=467630;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (ParquetDataset)...                                                                   

[03/21/24 14:53:17] INFO     Loading data from oa.data_processing.depth.no_mesh.3.intermediate  ]8;id=893572;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=501127;file:///opt/homebrew/Caskroom/miniforge/base/envs/af/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (ParquetDataset)...                                                                   

In [3]:
# Combine OA papers that cite AF levels 0-3
af_oa_data = pd.concat([level_0_primary, level_1_primary, level_2_primary, level_3_primary], axis=0)

In [4]:
# Display af_oa_data
af_oa_data

,parent_id,id,level,doi,publication_date,mesh_terms,cited_by_count
0,W3177828909,W3211795435,0,10.1093/nar/gkab1061,2021-11-17,"[[D030562, Databases, Protein], [D017510, Prot...",3437
1,W3177828909,W4281790889,0,10.1038/s41592-022-01488-1,2022-05-30,"[[D017510, Protein Folding], [D012984, Softwar...",3021
2,W3177828909,W3183475563,0,10.1038/s41586-021-03828-1,2021-07-22,"[[D019295, Computational Biology], [D000077321...",1688
3,W3177828909,W3202105508,0,10.1101/2021.10.04.463034,2021-10-04,None,1241
4,W3177828909,W4206153788,0,10.1016/j.cell.2021.12.046,2022-02-01,None,775
...,...,...,...,...,...,...,...
0,W4392058636,W4392041217,3,10.1103/physrevlett.132.080402,2024-02-22,None,0
0,W4392161883,W4392161867,3,10.1080/17499518.2024.2318849,2024-02-25,None,0
0,W4392192985,W4392198228,3,10.1038/s42255-024-01013-y,2024-02-27,None,0
0,W4392198237,W4392198357,3,10.1038/s41588-023-01646-x,2024-02-27,None,0


In [5]:
# Create list of dois of OA papers that cite AF levels 0-3 to match on
af_oa_dois = af_oa_data.doi.to_list()

In [ ]:
# Load icite data in chunks to find matches using doi to OA papers that cite AF levels 0-3
doi_matches = []

for chunk in pd.read_csv("/Users/jr/Downloads/icite/icite_metadata.csv", chunksize=1_000_000):
    current_matches = chunk.query(f"doi in {af_oa_dois}")
    doi_matches.append(current_matches)

In [ ]:
# Combine icite papers that cite AF levels 0-3
icite_matches = pd.concat(doi_matches)

In [23]:
# Check how many papers have been lost from OA to icite
print(f"{round(len(icite_matches) / len(af_oa_data) * 100, 2)}% papers matched on doi")

24.57% papers matched on doi


In [36]:
icite_matches

,pmid,doi,title,authors,year,journal,is_research_article,citation_count,field_citation_rate,expected_citations_per_year,citations_per_year,relative_citation_ratio,nih_percentile,human,animal,molecular_cellular,x_coord,y_coord,apt,is_clinical,cited_by_clin,cited_by,references,provisional,last_modified,num_cited_by_clin
31670737,33129791,10.1016/j.chest.2020.10.054,Impact of Corticosteroids in Coronavirus Disea...,"Edison J Cano, Xavier Fonseca Fuentes, Cristin...",2021,Chest,True,122,8.269781,2.932214,40.666667,13.87,98.9,0.50,0.0,0.50,-0.433013,0.25,0.95,False,33422626 34153729 34139335 35598005 34535158 3...,35880006 36819444 36243331 35136768 35764344 3...,32396996 32619760 32390367 32582740 32075786 3...,No,"01/28/2024, 12:07:34",12
31961549,33422642,10.1016/j.cct.2021.106272,Remdesivir for the treatment of COVID-19: A sy...,"Ahmad Al-Abdouh, Anas Bizanti, Mahmoud Barbara...",2021,Contemp Clin Trials,True,49,9.922650,3.476761,16.333333,4.70,92.5,0.50,0.0,0.50,-0.433013,0.25,0.95,False,34596223 34215210,35357033 34331674 34596223 34634340 35190252 3...,32338708 31774950 32445440 9784470 21195583 32...,No,"01/28/2024, 12:15:56",2
32049827,33511167,10.12998/wjcc.v9.i1.1,Necessary problems in re-emergence of COVID-19.,"Si Chen, Lin-Zhu Ren, Hong-Sheng Ouyang, Shen ...",2021,World J Clin Cases,False,2,9.326009,3.280194,0.666667,0.20,10.3,0.67,0.0,0.33,-0.288675,0.50,0.25,False,NaN,34514087 36051125,32105304 32079150 32645295 33031764 21576510 3...,No,"01/28/2024, 12:19:58",0
32057508,33518856,10.1016/j.molliq.2021.115298,A computational approach for the screening of ...,"Juhi Saraswat, Prashant Singh, Rajan Patel",2021,J Mol Liq,True,19,5.292914,1.951466,6.333333,3.25,86.7,0.00,0.0,1.00,-0.866025,-0.50,0.05,False,NaN,34847746 35594413 38250392 36605638 36530229 3...,35558908 23360222 29718510 33102687 29920814 3...,No,"01/28/2024, 12:20:25",0
32083889,33545393,10.1016/j.meegid.2021.104752,Probing SARS-CoV-2 sequence diversity of Pakis...,"Zaira Rehman, Massab Umair, Aamer Ikram, Afree...",2021,Infect Genet Evol,True,0,4.001206,1.525905,0.000000,0.00,0.0,0.50,0.0,0.50,-0.433013,0.25,0.05,False,NaN,NaN,21593585 25614110 25371430 32015507 31852899 3...,No,"01/28/2024, 12:21:52",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36834647,38310367,10.1002/mds.29724,Challenges and Future of the International Par...,"Kristina Kulcsarova, Matej Skorvanek",2024,Mov Disord,True,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,0.000000,-0.75,0.00,False,NaN,NaN,NaN,No,"02/05/2024, 05:43:28",0
36834723,38310443,10.1021/acs.jproteome.3c00654,Integrative Molecular Structure Elucidation an...,"Samuele Sala, Philipp Nitschke, Reika Masuda, ...",2024,J Proteome Res,True,0,NaN,NaN,NaN,NaN,NaN,0.50,0.0,0.50,0.000000,-0.75,0.00,False,NaN,NaN,NaN,No,"02/05/2024, 05:43:27",0
36834753,38310473,10.1063/5.0175756,How close are the classical two-body potential...,"Zheng Yu, Ajay Annamareddy, Dane Morgan, Bu Wang",2024,J Chem Phys,True,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,0.000000,-0.75,0.00,False,NaN,NaN,NaN,No,"02/05/2024, 05:43:27",0
36834756,38310476,10.1063/5.0182524,Surveying the energy landscape of coarse-grain...,"Katherine M Kidder, M Scott Shell, W G Noid",2024,J Chem Phys,True,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,1.00,0.000000,-0.75,0.00,False,NaN,NaN,NaN,No,"02/05/2024, 05:43:27",0


In [34]:
# Add column for number of times cited by clinical trial
icite_matches['num_cited_by_clin'] = (
    icite_matches.cited_by_clin
    .str.count(' ')
    .add(1)
    .fillna(0, downcast='infer')
)

In [71]:
from Bio import Entrez
from typing import Optional, List
import logging
from datetime import datetime

def convert_month_abbreviation_to_number(month_abbr: str) -> str:
    """
    Convert a three-letter month abbreviation to a two-digit month number.

    Args:
        month_abbr (str): A three-letter abbreviation of a month (e.g., 'Jan', 'Feb', etc.)

    Returns:
        str: The two-digit month number corresponding to the abbreviation.
    """
    if month_abbr == 'XX':
        return 'XX'
    return datetime.strptime(month_abbr, '%b').strftime('%m')


def get_publication_date(pmid: str) -> str:
    """
    Fetches the publication date of a PubMed article using its PMID.
    Returns np.nan if the date cannot be retrieved.

    Args:
        pmid (str): The PubMed ID of the article.

    Returns:
        str: The publication date of the article in YYYY-MM-DD format, or np.nan.
    """
    try:
        handle = Entrez.efetch(db="pubmed", id=pmid, retmode="xml")
        records = Entrez.read(handle)
        handle.close()
 
        date = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
        year = date.get('Year', 'XXXX')
        month = convert_month_abbreviation_to_number(date.get('Month', 'XX'))
        day = date.get('Day', 'XX')
        return f"{year}-{month}-{day}"
    except Exception:
        return np.nan


def get_pub_dates_for_pmids(pmids: str) -> List[str]:
    """
    Fetches publication dates for a list of PubMed IDs.
    Skips if the input is NaN or non-string.

    Args:
        pmids (str): A string containing PubMed IDs separated by spaces.

    Returns:
        List[str]: A list of publication dates in YYYY-MM-DD format or np.nan.
                  Returns an empty list if pmids is NaN or non-string.
    """
    if pd.isna(pmids) or not isinstance(pmids, str):
        return np.nan
    return [get_publication_date(pmid) for pmid in pmids.split()]

In [72]:
# Example usage
pmid = "33422626"
print(get_publication_date(pmid))

2021-05-XX


In [73]:
get_pub_dates_for_pmids('33422626 34153729 34139335 35598005 34535158 35907774 33634996 36645234 35488261 34493750 36272908 35727348')


[
    '2021-05-XX',
    '2021-08-XX',
    '2021-10-XX',
    '2022-05-21',
    '2021-09-17',
    '2023-02-XX',
    '2021-02-XX',
    '2023-09-01',
    '2022-04-29',
    '2021-09-07',
    '2023-01-XX',
    '2022-07-XX'
]

In [59]:
# Add clinial article publication dates

In [ ]:
# Calculate how many have missing info

In [ ]:
# Find earliest clinical article publication date

In [25]:
# Create list of dois of OA papers that cite AF levels 0-3 to match on (separate for each level)
af_oa_dois_0 = af_oa_data.query("level == 0").doi.to_list()
af_oa_dois_1 = af_oa_data.query("level == 1").doi.to_list()
af_oa_dois_2 = af_oa_data.query("level == 2").doi.to_list()
af_oa_dois_3 = af_oa_data.query("level == 3").doi.to_list()

In [ ]:
# Load icite data in chunks to find matches using doi to OA papers that cite AF levels 0-3 (separately)
doi_matches_0 = []
doi_matches_1 = []
doi_matches_2 = []
doi_matches_3 = []

for chunk in pd.read_csv("/Users/jr/Downloads/icite/icite_metadata.csv", chunksize=1_000_000):
    current_matches_0 = chunk.query(f"doi in {af_oa_dois_0}")
    doi_matches_0.append(current_matches_0)
    
    current_matches_1 = chunk.query(f"doi in {af_oa_dois_1}")
    doi_matches_1.append(current_matches_1)

    current_matches_2 = chunk.query(f"doi in {af_oa_dois_2}")
    doi_matches_2.append(current_matches_2)

    current_matches_3 = chunk.query(f"doi in {af_oa_dois_3}")
    doi_matches_3.append(current_matches_3)

In [27]:
# Combine icite papers that cite AF levels 0-3 (separately)
icite_matches_0 = pd.concat(doi_matches_0)
icite_matches_1 = pd.concat(doi_matches_1)
icite_matches_2 = pd.concat(doi_matches_2)
icite_matches_3 = pd.concat(doi_matches_3)

In [28]:
# Check how many papers have been lost from OA to icite
print(f"{round(len(icite_matches_0) / len(af_oa_dois_0) * 100, 2)}% papers matched on doi level 0")
print(f"{round(len(icite_matches_1) / len(af_oa_dois_1) * 100, 2)}% papers matched on doi level 1")
print(f"{round(len(icite_matches_2) / len(af_oa_dois_2) * 100, 2)}% papers matched on doi level 2")
print(f"{round(len(icite_matches_3) / len(af_oa_dois_3) * 100, 2)}% papers matched on doi level 3")

65.61% papers matched on doi level 0
39.63% papers matched on doi level 1
37.77% papers matched on doi level 2
33.19% papers matched on doi level 3
